# 再学習プロセス

In [4]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.metrics import f1_score, accuracy_score
from datetime import datetime
from pathlib import Path
from mlflow.tracking import MlflowClient

import sys
sys.path.append('../..')

from src.utils.io import load_month_data
from src.utils.preprocess import preprocess_pipeline

In [12]:
# 設定値を辞書で管理

CONFIG = {
    "data_info": [2014, 4],
    "model_name": "citibike_membership_model",
    "eval_threshold": 0.80,   # F1スコアの再学習トリガー
    "experiment_name": "citibike_retraining",
}

In [22]:
# 最新モデルをMLflow Registryから取得

client = MlflowClient()

try:
    latest_model = client.get_model_version_by_alias(
        name=CONFIG["model_name"],
        alias="production"
    )

    print(f"エイリアスでバージョン情報を取得しました。")
    print(f"Loaded model: {latest_model.name} (v{latest_model.version})")

    model_uri = f"models:/{CONFIG['model_name']}@production"
    
    model = mlflow.pyfunc.load_model(model_uri)
    
    print(f"モデルを正常にロードしました。URI: {model_uri}")

except Exception as e:
    print(f"モデルの取得またはロード中にエラーが発生しました: {e}")


エイリアスでバージョン情報を取得しました。
Loaded model: citibike_membership_model (v2)
モデルを正常にロードしました。URI: models:/citibike_membership_model@production


In [13]:
# データの読み込み、整形

df_raw = load_month_data(*CONFIG["data_info"])
df = preprocess_pipeline(df_raw)

X = df.drop("is_member", axis=1)
y = df["is_member"]

print(f"Data loaded: {X.shape[0]} samples, {X.shape[1]} features")

Loading: [PosixPath('/app/data/raw/2014-citibike-tripdata/4_April/201404-citibike-tripdata_1.csv')]
Data loaded: 670780 samples, 7 features


In [24]:
latest_model.tags

{'source_run_id': '441fde457cdc427dad211185c3d84bdb'}

In [15]:
df["is_member"].value_counts()

is_member
1    607733
0     63047
Name: count, dtype: int64

In [16]:
# 部分サンプリング（高速化）
sample = X.sample(frac=0.2, random_state=42)
y_sample = y.loc[sample.index]

y_pred = model.predict(sample)

current_f1 = f1_score(y_sample, y_pred)
current_acc = accuracy_score(y_sample, y_pred)

print(f"Current model performance: F1={current_f1:.3f}, Acc={current_acc:.3f}")

Current model performance: F1=1.000, Acc=1.000


In [ ]:
if current_f1 < CONFIG["eval_threshold"]:
    print("Model performance below threshold — retraining triggered.")
    trigger_retrain = True
else:
    print("Model performance acceptable — retraining skipped.")
    trigger_retrain = False

✅ Model performance acceptable — retraining skipped.


## run_idの管理

現状では、Modelsの中でrun_idが取得できないので、  
実験中とは別でrun_idを持ったままモデルを登録できる方法を探す

In [19]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

run_id = "a524d0163c8c4f72855e5dcb53ed99e9"  # ← UIのSource Run IDを貼る
run = client.get_run(run_id)

print(run.data.params)
print(run.info.artifact_uri)

{'n_estimators': '100', 'learning_rate': '0.1', 'max_depth': '5'}
/app/mlruns/1/a524d0163c8c4f72855e5dcb53ed99e9/artifacts


In [25]:
# 手動でtagとしてrun_idを添付できる

client.set_model_version_tag(
    name="citibike_membership_model",
    version=5,
    key="source_run_id",
    value="a524d0163c8c4f72855e5dcb53ed99e9"  # UIで見えるSource Run IDをここに
)

In [26]:
client.set_model_version_tag(
    name="citibike_membership_model",
    version=2,
    key="source_run_id",
    value="441fde457cdc427dad211185c3d84bdb"  # UIで見えるSource Run IDをここに
)

In [32]:
# 対象Experiment内のRunを一覧取得

import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

# 対象Experiment名を指定
experiment_name = "citibike_membership"
experiment = client.get_experiment_by_name(experiment_name)

runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.test_f1_score DESC"],
)


In [33]:
runs

[<Run: data=<RunData: metrics={'test_accuracy': 1.0,
  'test_f1_score': 1.0,
  'test_precision': 1.0,
  'test_recall': 1.0,
  'train_accuracy': 0.9999375200456521,
  'train_f1_score': 0.9999679880488715}, params={'max_iter': '500', 'random_state': '42'}, tags={'data_source': '[2014, 1]',
  'framework': 'sklearn',
  'mlflow.runName': 'logistic_regression_20251022_122051',
  'mlflow.source.name': '/usr/local/lib/python3.11/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'appuser',
  'model_type': 'logistic_regression'}>, info=<RunInfo: artifact_uri='/app/mlruns/1/aff718a4185e46b1a8843d8b3343800f/artifacts', end_time=1761135656809, experiment_id='1', lifecycle_stage='active', run_id='aff718a4185e46b1a8843d8b3343800f', run_name='logistic_regression_20251022_122051', start_time=1761135651646, status='FINISHED', user_id='appuser'>, inputs=<RunInputs: dataset_inputs=[<DatasetInput: dataset=<Dataset: digest='0b63bd9d', name='citibike_data_20251022', prof

In [37]:
best_run_id = runs[0].info.run_id

In [40]:
# モデルの保存場所を確認（artifacts配下の model/）
model_uri = f"runs:/{best_run_id}/model"
model_uri

'runs:/aff718a4185e46b1a8843d8b3343800f/model'

In [41]:
# Model Registryに登録
result = mlflow.register_model(
    model_uri=model_uri,
    name="citibike_membership_model"
)

print(f"Registered as {result.name} (version={result.version})")

Registered model 'citibike_membership_model' already exists. Creating a new version of this model...
2025/10/23 08:03:51 WARNING mlflow.tracking._model_registry.fluent: Run with id aff718a4185e46b1a8843d8b3343800f has no artifacts at artifact path 'model', registering model based on models:/m-57d3530efb6e4f53bd3a8e6f4e842363 instead
2025/10/23 08:03:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: citibike_membership_model, version 6


Registered as citibike_membership_model (version=6)


Created version '6' of model 'citibike_membership_model'.


In [42]:
client.set_model_version_tag(
    name="citibike_membership_model",
    version=result.version,
    key="registered_from_run",
    value=best_run_id
)

client.set_model_version_tag(
    name="citibike_membership_model",
    version=result.version,
    key="comment",
    value="Top F1 model from experiment citibike_membership"
)

In [47]:

latest_model = client.get_model_version_by_alias(
    name=CONFIG["model_name"],
    alias="production"
)

latest_production_version = latest_model.version

In [50]:


# もしProductionエイリアスが付いているバージョンが存在すれば、エイリアスを削除
if latest_production_version:
    # get_latest_versionsはリストを返すため、最初のエントリを使用
    # current_prod_version_number = latest_production_version[0].version
    
    # 既存のProductionエイリアスを削除
    client.delete_registered_model_alias(
        name="citibike_membership_model",
        alias="production"
    )


# 2. 新しいモデルバージョンに 'Production' エイリアスを付与

# result.version は新しいモデルのバージョン番号
client.set_registered_model_alias(
    name="citibike_membership_model",
    alias="production",
    version=result.version # 新しくProductionにするバージョン番号
)

## register_best_modelの動作確認


In [5]:
from src.pipelines.register_best_model import register_best_model

In [3]:
register_best_model()

Searching best run from experiment 'citibike_membership'...
Best run: aff718a4185e46b1a8843d8b3343800f (metric=test_f1_score: 1.0000)
Current Production model: v8 (run_id=aff718a4185e46b1a8843d8b3343800f)
✅ Best run is same as current Production model. No update needed.


## 再学習パイプライン検討

新しいデータが来たときに：
- 現行のProductionモデルをロード
- 新データに対して推論 → 精度・分布などを比較
- 条件（精度改善・閾値超えなど）を満たせば再学習
- MLflowに自動でログ＋Model Registry更新